In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from scipy.optimize import minimize
import gurobipy as gp
from gurobipy import GRB
import time


In [2]:
# 価格を生成する関数
def create_price(r_min: float, r_max: float, M: int) -> NDArray[np.float_]:
    # r_minとr_maxの間のランダムな0.1刻みの少数をM個生成
    price = np.random.uniform(r_min, r_max, M)
    price = np.round(price, 1)

    return price


# alphaを作成する関数
def alpha_star(M: int) -> NDArray[np.float_]:
    alpha_star = np.random.uniform(M, 3 * M, size=M)
    return alpha_star


# betaを作成する関数
def beta_star(M: int, M_prime: int) -> NDArray[np.float_]:
    beta_star = np.zeros((M, M_prime))

    for m in range(M):
        for m_prime in range(M_prime):
            if m == m_prime:
                beta_star[m, m_prime] = np.random.uniform(-2 * M, -M)
            else:
                beta_star[m, m_prime] = np.random.uniform(0, 1)

    return beta_star


def quantity_function(
    price: NDArray[np.float_],
    alpha: NDArray[np.float_],
    beta: NDArray[np.float_],
    delta: float = 0.1,  # ノイズレベルを指定（例として0.1を使用）
) -> list[float]:
    M = len(price)
    quantity_list = []
    q_m_no_noise = []

    # ステップ1: ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]
        quantity = alpha[m] + sum_beta
        q_m_no_noise.append(quantity)

    # E[q_m^2]を計算
    E_q_m_squared = np.mean(np.array(q_m_no_noise) ** 2)

    # ステップ2: ノイズの標準偏差sigmaを計算
    sigma = delta * np.sqrt(E_q_m_squared)

    # ステップ3: ノイズを加えて最終的なq_mを計算
    for m in range(M):
        epsilon = np.random.normal(0, sigma)
        quantity = q_m_no_noise[m] + epsilon
        quantity_list.append(quantity)

    return quantity_list


def sales_function(
    price: NDArray[np.float_], alpha: NDArray[np.float_], beta: NDArray[np.float_]
) -> list[float]:
    M = len(price)
    sales_list = []

    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]

        quantity = alpha[m] + sum_beta
        sales_list.append(quantity * price[m])

    return sales_list


In [3]:
# 目的関数を定義（最大化問題を最小化問題に変換）
def sales_objective_function(prices, alpha, beta, M):
    return -sum(
        prices[m] * (alpha[m] + sum(beta[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


In [4]:
def sales_optimize(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices


In [5]:
# 目的関数を定義
def predict_objective_function(
    prices: NDArray[np.float_], intercepts: [float], coefs: [NDArray[np.float_]], M: int
) -> float:
    # 各変数の内容をデバッグ出力
    # print("prices:", prices)
    # print("intercepts:", intercepts)
    # print("coefs:", coefs)
    # print("M:", M)

    return -sum(
        prices[m]
        * (intercepts[m] + sum(coefs[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


In [16]:
# 予測と最適化を行う関数
def lb_predict_optimize(
    lb: list[float], M: int, X: NDArray[np.float_], Y: NDArray[np.float_]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(lb[m], 1.0) for m in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return -optimal_value


In [36]:
M = 20
alpha = alpha_star(M)
beta = beta_star(M, M)
r_m = 0.6
r_M = 1.0
# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)

X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)

z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]


In [29]:
result_bounds = minimize(
    lb_predict_optimize,
    lb,
    args=(M, X, y),
    bounds=[(0.6, 1.0) for _ in range(M)],
    method="Nelder-Mead",
    options={"adaptive": True},
)


In [27]:
result_bounds.x

array([ 0.60022287,  0.59879883,  0.6011289 ,  0.60025579,  0.59982033,
        0.59931362, -0.3999958 ,  0.60041439,  0.60095604,  0.60134882])

In [28]:
result_bounds.fun

-91.37063752589485

In [30]:
result_bounds.x

array([0.6552004 , 0.64478683, 0.6       , 0.75544539, 0.60300609,
       0.6246599 , 0.6       , 0.60308886, 0.61149259, 0.60077402])

In [31]:
result_bounds.fun

-91.36214143838058

In [32]:
def lb_estimate_Nelder(lb, M, X, y):
    result_bounds = minimize(
        lb_predict_optimize,
        lb,
        args=(M, X, y),
        bounds=[(0.6, 1.0) for _ in range(M)],
        method="Nelder-Mead",
        options={"adaptive": True},
    )
    return result_bounds.x, -result_bounds.fun


In [33]:
def lb_estimate_BFGS(lb, M, X, y):
    result_bounds = minimize(
        lb_predict_optimize,
        lb,
        args=(M, X, y),
        bounds=[(0.6, 1.0) for _ in range(M)],
        method="L-BFGS-B",
    )
    return result_bounds.x, -result_bounds.fun


In [37]:
lb_estimate_Nelder(lb, M, X, y)

(array([0.60034233, 0.60223527, 0.60097341, 0.60020894, 0.60341841,
        0.60195544, 0.60015346, 0.6       , 0.60347215, 0.60037698,
        0.62079403, 0.60084868, 0.60400414, 0.60042754, 0.60123831,
        0.61984668, 0.60202102, 0.60178287, 0.60392014, 0.60016279]),
 465.3369952147665)

In [38]:
lb_estimate_BFGS(lb, M, X, y)

(array([0.60000028, 0.60000022, 0.60000019, 0.60000018, 0.60000015,
        0.60000013, 0.60000016, 0.6       , 0.60000006, 0.60000043,
        0.60000031, 0.6000003 , 0.60000015, 0.60000023, 0.60000006,
        0.6       , 0.60000032, 0.6       , 0.6000002 , 0.60000012]),
 465.33699521463467)